In [1]:
# Setting up the environment

import svelte_widget
import xgboost
import shap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

2023-05-19 14:12:06.218623: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-19 14:12:06.347799: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-19 14:12:06.348738: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 14:12:07.373835: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Upload and set up data

from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("diabetes_data.csv")

scaler = MinMaxScaler()

df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

X = df.drop(columns=['Outcome'])
y = df['Outcome']

In [3]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333,1.0
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667,0.0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1.0
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000,0.0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000,1.0
...,...,...,...,...,...,...,...,...,...
763,0.588235,0.507538,0.622951,0.484848,0.212766,0.490313,0.039710,0.700000,0.0
764,0.117647,0.613065,0.573770,0.272727,0.000000,0.548435,0.111870,0.100000,0.0
765,0.294118,0.608040,0.590164,0.232323,0.132388,0.390462,0.071307,0.150000,0.0
766,0.058824,0.633166,0.491803,0.000000,0.000000,0.448584,0.115713,0.433333,1.0


In [4]:
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Defining and training two Neural Networks

import tensorflow as tf
from tensorflow import keras

# Define the first neural network model
model_1 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_1.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Define the second neural network model
model_2 = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_2.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

2023-05-19 14:12:08.976418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-19 14:12:08.977451: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10
20/20 [==============================] - 1s 14ms/step - loss: 0.6819 - accuracy: 0.5912 - val_loss: 0.6656 - val_accuracy: 0.6429
Epoch 2/10
20/20 [==============================] - 0s 5ms/step - loss: 0.6583 - accuracy: 0.6531 - val_loss: 0.6533 - val_accuracy: 0.6429
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 0.6449 - accuracy: 0.6531 - val_loss: 0.6420 - val_accuracy: 0.6429
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.6332 - accuracy: 0.6515 - val_loss: 0.6321 - val_accuracy: 0.6688
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6199 - accuracy: 0.6612 - val_loss: 0.6190 - val_accuracy: 0.6429
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6041 - accuracy: 0.6857 - val_loss: 0.6077 - val_accuracy: 0.6623
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5884 - accuracy: 0.6808 - val_loss: 0.5962 - val_accuracy: 0.6688
Epoch 8/10
20/20 [=

In [6]:
z = svelte_widget.ExampleWidget(X_test, y_test, model_1.predict, model_2.predict)
z

Start...
Predictions obtained
5/5 [==============================] - 0s 3ms/step
Shaps now
Shaps XGB
Shaps NN
Calculating...
-> m1
29/29 [==============================] - 0s 1ms/step


Exact explainer:   4%|██▉                                                                       | 6/154 [00:00<?, ?it/s]

28/28 [==============================] - 0s 1ms/step


Exact explainer:   5%|███▍                                                              | 8/154 [00:11<01:36,  1.52it/s]

27/27 [==============================] - 0s 2ms/step


...

Exact explainer: 100%|████████████████████████████████████████████████████████████████| 154/154 [03:15<00:00,  1.27s/it]

28/28 [==============================] - 0s 1ms/step


Exact explainer: 155it [03:16,  1.34s/it]                                                                               

Difference...
Done


ExampleWidget(dataset={'Pregnancies': [0.3529411764705882, 0.11764705882352941, 0.11764705882352941, 0.4705882…